<font size=5, font color='blue'> stata_df is the dataframe downloaded through Stata API

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None

stata_df = pd.read_stata("FRANC_ZONE.dta")
stata_df.head()

,period,period_start_day,value,frequency,dataset_code,dataset_name,country,freq,indicator,indexed_at,provider_code,series_code,series_name,series_num,_frequency
0,1991,1991-01-01,-5.6,annual,FRANCZONE,Statistical series of member countries from mo...,BEN,A,budget_balance_percent,2022-04-04T22:41:53.855Z,Franc-zone,budget_balance_percent.BEN.A,"Fiscal balance, commitment basis including gra...",Master,
1,1991,1991-01-01,1.0,annual,FRANCZONE,Statistical series of member countries from mo...,BEN,A,price_index_percent,2022-04-04T22:41:53.855Z,Franc-zone,price_index_percent.BEN.A,Change in the consumer price index (annual ave...,Master,
2,1991,1991-01-01,535.5,annual,FRANCZONE,Statistical series of member countries from mo...,BEN,A,gdp_FCFA,2022-04-04T22:41:53.855Z,Franc-zone,gdp_FCFA.BEN.A,Nominal GDP (in billions of FCFA) - Benin - An...,Master,
3,1991,1991-01-01,151.2,annual,FRANCZONE,Statistical series of member countries from mo...,BEN,A,money_FCFA,2022-04-04T22:41:53.855Z,Franc-zone,money_FCFA.BEN.A,Broad money (in billions of FCFA) - Benin - An...,Master,
4,1991,1991-01-01,14.5,annual,FRANCZONE,Statistical series of member countries from mo...,BEN,A,investment,2022-04-04T22:41:53.855Z,Franc-zone,investment.BEN.A,Investment rate (% of GDP) - Benin - Annually,Master,


<font size=5, font color='blue'> python_df is the dataframe downloaded through Python API. 

In [2]:
python_df = pd.read_pickle("FRANC_ZONE.pkl")
python_df.head()

,@frequency,provider_code,dataset_code,dataset_name,series_code,series_name,original_period,period,original_value,value,indicator,country,freq,Economic indicator,Country,Frequency
0,annual,Franc-zone,FRANCZONE,Statistical series of member countries from mo...,budget_balance_percent.BEN.A,"Fiscal balance, commitment basis including gra...",1991,1991-01-01,-5.6,-5.6,budget_balance_percent,BEN,A,"Fiscal balance, commitment basis including gra...",Benin,Annually
435,annual,Franc-zone,FRANCZONE,Statistical series of member countries from mo...,gdp_FCFA.BEN.A,Nominal GDP (in billions of FCFA) - Benin - An...,1991,1991-01-01,535.5,535.5,gdp_FCFA,BEN,A,Nominal GDP (in billions of FCFA),Benin,Annually
870,annual,Franc-zone,FRANCZONE,Statistical series of member countries from mo...,investment.BEN.A,Investment rate (% of GDP) - Benin - Annually,1991,1991-01-01,14.5,14.5,investment,BEN,A,Investment rate (% of GDP),Benin,Annually
1305,annual,Franc-zone,FRANCZONE,Statistical series of member countries from mo...,money_FCFA.BEN.A,Broad money (in billions of FCFA) - Benin - An...,1991,1991-01-01,151.2,151.2,money_FCFA,BEN,A,Broad money (in billions of FCFA),Benin,Annually
1740,annual,Franc-zone,FRANCZONE,Statistical series of member countries from mo...,price_index_percent.BEN.A,Change in the consumer price index (annual ave...,1991,1991-01-01,1.0,1.0,price_index_percent,BEN,A,Change in the consumer price index (annual ave...,Benin,Annually


<font size=5, font color='blue'> Check whether the two dataframes have the same number of rows, which means equal number of data points / observations. Column numbers of the two dataframes may differ due to the difference between Stata and Python APIs, and so do the order, datatypes and names of columns and the order of rows, which is why we don't require a complete match between python_df and stata_df.

In [3]:
stata_df.shape[0]==python_df.shape[0]

True

<font size=5, font color='blue'> ['period_start_day'] in stata_df is the most accurate timestamp assigned to each observation (row). In python_df its equivalent is ['period'], so we rename it to match the column name in stata_df. We match the datatype of the two columns by converting both of them to pandas datetime objects. We also convert the datatype of ['value'] in stata_df to numeric.

In [4]:
python_df.rename(columns={'period':'period_start_day'},inplace=True)
stata_df['period_start_day'] = pd.to_datetime(stata_df['period_start_day'])
python_df['period_start_day'] = pd.to_datetime(python_df['period_start_day'])
stata_df['value'] = pd.to_numeric(stata_df['value'],'coerce')

<font size=5, font color='blue'> Check whether ['period_start_day', 'series_code'] can identify a unique row in both stata_df and python_df, if not then there is a duplication or error in Dbnomics.

In [5]:
cols = ['period_start_day', 'series_code']
sum(stata_df.duplicated(subset=cols))+sum(python_df.duplicated(subset=cols)) == 0

True

<font size=5, font color='blue'> Then we sort the two dataframes by ['period_start_day', 'series_code']. This ensures observations in the two dataframes are in the same order. Finally we check whether the two sub-frames [['period_start_day','series_code','value']] sliced from stata_df and python_df are exactly the same.

In [6]:
stata_sub = stata_df.sort_values(by=cols)[cols+['value']].reset_index(drop=True)
python_sub = python_df.sort_values(by=cols)[cols+['value']].reset_index(drop=True)
stata_sub.equals(python_sub)

True